<a href="https://colab.research.google.com/github/laffertybrian/Math-Skill-Builder/blob/main/Testing_for_Little_Professor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile professor.py

import random, csv, os, time, datetime, sys
import pandas as pd

# variables for file storage
game_results_storage = "little_professor_game_resutls.csv"
player_settings_storage = "player_settings.csv"
FOLDER = "/content/drive/MyDrive/Data Science Projects/Math_Skill_Builder/Data/"

# creating folder for storage
os.makedirs(FOLDER, exist_ok = True)

game_results_fieldnames = ['Player_Name', 'First_Number', 'Second_Number', 
                           'Solution', 'Player_Answer', 'Operator', 'Seconds', 
                           'Correct','Testing', 'Date', 'Prediction']
player_settings_fieldnames = ['Player_Name', 'Games_Played', 'Level', 'Focus', 
                              'Rounds', 'Operator']

def main():

  game_mode = commandline()

  if game_mode == 'testing':
    testing_mode()

  if game_mode == 'normal':


    welcome()

    final_score = 0

    for _ in range(player.rounds):
        details = create_problem(player.level, player.focus, player.operator)
        record_to_file(game_results_storage, problem.export(), game_results_fieldnames)
        score = check_solution(*details)
        final_score += score

    print(f'Score: {final_score}')

def welcome():
  global player
  print('Welcome to Little Professor! A math skill building game.')
  player_name = input('Enter your player name, or if you are new here type New: ')

  if player_name.lower() == 'new':
    player = Player.get()
    record_to_file(player_settings_storage, player.export(), player_settings_fieldnames)
    return

  if file_check(player_settings_storage):
    df = retrieve_from_file(player_settings_storage)
    if player_name in df['Player_Name'].tolist():
      print("Welcome Back")
      used_last_df = retrieve_from_file(player_settings_storage)
      player_set = used_last_df[used_last_df['Player_Name'] == player_name].sort_index(ascending = False)    

      sorted_dict = player_set.to_dict('records')
      converted = number_convert(sorted_dict[0])
      while True:
        used_last = input('Would you like to use the settings from your last game? (Y or N)')
        if used_last.lower() == 'y':
          player = Player(**converted)
          update_dataframe(player.export())
          return
        if used_last.lower() == 'n':
          player = Player(Player_Name = player_name, Games_Played = converted['Games_Played'])
          player.reset_settings()
          update_dataframe(player.export())
          return

    else:
      no_name()

  else:
    no_name()

def no_name():
  print("We did not recognize your name. Let me ask you some questions and get you started!")
  global player
  player = Player.get()
  record_to_file(player_settings_storage, player.export(), player_settings_fieldnames)

def get_name():
  while True:
    name = input('Name: ')

    if file_check(player_settings_storage):
      df = retrieve_from_file(player_settings_storage)

      if name in df['Player_Name'].tolist():
        print('Another player has that name, choose another.')
      else:
        return name
    else:
      return name


def get_level():
    print('Type 1 for single, 2 for double, or 3 for triple digit numbers!')
    print('Or 4 to customize your number range.')
    start_settings = []
    while True:
        try:
            level = int(input('Level: '))
            if level in [1, 2, 3, 4]:
              break
        except ValueError:
            pass
    start_settings.append(level)            
    if start_settings[0] == 4:
        while True:
          try:
            num_range = input('Number range X-Y: ').split('-')
            cus_x = int(num_range[0])
            start_settings.append(cus_x)
            cus_y = int(num_range[1])
            start_settings.append(cus_y)
            break
          except ValueError:
            pass
    return start_settings

def get_focus():
  focus = []
  while True:
    focus_check = input("Would you like to focus on specific numbers? (Y or N) ")
    if focus_check.lower() == 'y':
      while True:
        try:
          numbers = input("What number(s) would you like to focus on? (ex. 1,3,5) ").split(',')
          for num in numbers:
            num = num.strip(' ')
            focus.append(int(num))
          return focus
        except:
          pass

    if focus_check.lower() == 'n':
      focus.append('na')
      return focus

def get_rounds():
  while True:
    try:
      rounds = int(input('How many questions would you like? '))
      return rounds
    except ValueError:
      pass

def get_operator():
  while True:
    try:
      operator = input("Pick your operator [+, -, *, /]: ")
      if operator in ['+', '-', '*', '/']:
        return operator
    except ValueError:
      pass


def generate_integer(level):
    if level[0] == 1:
        return random.randint(0, 9)
    if level[0] == 2:
        return random.randint(10, 99)
    if level[0] == 3:
        return random.randint(100, 999)
    if level[0] == 4:
        return random.randint(level[1], level[2])
    if level[0] == 5:
        return random.randint(0, 15)

def create_problem(level, focus, operator):
    global problem
    problem = Problem(Player_Name=player.name)

    x = generate_integer(level)
    if focus[0] == 'na':
      y = generate_integer(level)
    else:
      y = random.choice(focus)

    # filter to prevent y from becoming 0 when dividing
    while operator == '/' and y == 0:
      if focus[0] == 'na':
        y = generate_integer(level)
      else:
        y = random.choice(focus)

    if operator != '/':
      time_start = time.time()
      answer = ask_question(x,y,operator)
      time_stop = time.time()

      if operator == '+':
        ans = x + y
      elif operator == '-':
        ans = x - y
      elif operator == '*':
        ans = x * y

      problem.first_number = x
      problem.second_number = y
      problem.solution = ans
      problem.player_answer = answer
      problem.operator = operator
      problem.seconds = round(time_stop - time_start, 2)
      return [x, y, ans, answer, operator]
      
    else:
      time_start = time.time()
      answer = ask_question(x,y,operator)
      time_stop = time.time()

      product = x * y
      ans = x

      problem.first_number = product
      problem.second_number = y
      problem.solution = ans
      problem.player_answer = answer
      problem.operator = operator
      problem.seconds = round(time_stop - time_start, 2)

      return [product, y, ans, answer, operator]

def check_solution(first, second, solution, response, operator):
    tries = 0

    if solution == response:
        return 1
    else:
        while tries < 2:
            print('EEE')
            again = int(input(f'{first} {operator} {second} = '))
            if again == solution:
                return 0
            tries += 1

    print('EEE')
    print(f'Correct answer: {first} {operator} {second} = {solution}')
    return 0

def ask_question(x,y,sign):
  if sign != '/':
    while True:
      try:
        user_answer = int(input(f"{x} {sign} {y} = "))
        return user_answer

      except ValueError:
        print("Response must be numerical")

  else:
    while True:
      try:
        user_answer = int(input(f"{x * y} {sign} {y} = "))
        return user_answer
        
      except ValueError:
        print("Response must be numerical")


# function to convert strings to integers after retrieval from dataframe

def number_convert(dict_):
  converted_level = []
  converted_focus = []
  convert = dict_

  level_list = dict_['Level']
  if dict_['Level'].startswith('['):
    level_list = level_list.strip('[')
    level_list = level_list.strip(']')
    level_list = level_list.split(',')

    for _ in level_list:
      _.replace(' ', '')
      converted_level.append(int(_))
  convert['Level'] = converted_level

  focus_list = dict_['Focus']
  if dict_['Focus'].startswith('['):
    focus_list = focus_list.strip('[')
    focus_list = focus_list.strip(']')
    focus_list = focus_list.strip("'")
    focus_list = focus_list.split(',')

    if focus_list == ['na']:
      convert['Focus'] = focus_list
    else:
      for _ in focus_list:
        _.replace(' ', '')
        converted_focus.append(int(_))
      convert['Focus'] = converted_focus

  round = dict_['Rounds']
  convert['Rounds'] = int(round)

  game = dict_['Games_Played']
  convert['Games_Played'] = int(game)

  return convert

# creating a function to check if a file exsits
def file_check(file):
  folderfile = FOLDER + file
  file_exists = os.path.isfile(folderfile)
  if file_exists:
    return True
  else:
    return False

def record_to_file(file, results, fieldnames, edit = 'a'):
  folderfile = FOLDER + file
  if file_check(file):
    with open(folderfile, edit, newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      if edit == 'w':
        writer.writeheader()
        for row in results:
          writer.writerow(row)
      else:
        writer.writerow(results)

  else:
    with open(folderfile, 'w', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writeheader()
      writer.writerow(results)

def retrieve_from_file(file):
  folderfile = FOLDER + file
  with open(folderfile, 'r') as f:
    reader = csv.DictReader(f)
    results_df = pd.DataFrame(reader)
  return results_df


def update_dataframe(info):
  df = retrieve_from_file(player_settings_storage)
  players_index = df.index[df['Player_Name'] == info['Player_Name']]
  index_num = list(players_index).pop(0)
  for key, value in info.items():
    df.at[index_num, key] = value

  df_dictionaries = df.to_dict('records')
  record_to_file(player_settings_storage, df_dictionaries, player_settings_fieldnames, 'w')


def commandline():
  """ This funciton with check the command line and determine if the normal game
  is played, or the testing mode. The function will return 'normal' or 'testing'
  based off command line arguments"""

  if len(sys.argv) == 1:
    return 'normal'
  elif len(sys.argv) == 2:
    if sys.argv[1] == 'testing':
      return 'testing'
    else:
      print('Incorrect Mode')
  else:
    print('Incorrect Argument(s)')


class Player:

  def __init__(self, Player_Name, Level = 0, Focus = 0, Rounds = 0, Operator = '', Games_Played = 0):
    self.name = Player_Name
    self.level = Level
    self.focus = Focus
    self.rounds = Rounds
    self.operator = Operator
    self.games_played = Games_Played + 1

  def __str__(self):
    ...
  
  @classmethod
  def get(cls):
    name = get_name()
    level = get_level()
    focus = get_focus()
    rounds = get_rounds()
    operator = get_operator()
    return cls(name, level, focus, rounds, operator)

  def reset_settings(self):
    self.level = get_level()
    self.focus = get_focus()
    self.rounds = get_rounds()
    self.operator = get_operator()

  def export(self):
    return {'Player_Name': self.name, 'Level': self.level, 'Focus': self.focus, 
            'Rounds': self.rounds, 'Operator': self.operator, 
            'Games_Played': self.games_played}

# creating a game results class to store and retrieve all information

class Problem:

  def __init__(self, Player_Name = None, First_Number = None, Second_Number = None, Solution = None,
               Player_Answer = None, Operator = None, Seconds = None, Correct = None,
               Testing = None, Prediction = None):
    self.player_name = Player_Name
    self.first_number = First_Number
    self.second_number = Second_Number
    self.solution = Solution
    self.player_answer = Player_Answer
    self.operator = Operator
    self.seconds = Seconds
    self.correct = Correct
    self.test = Testing
    self.prediction = Prediction


  def export(self):
    if self.solution == self.player_answer:
      if self.seconds < 7:
        self.correct = 'Yes'
      else:
        self.correct = 'No'
    else:
      self.correct = 'No'

    self.date = datetime.date.today().isoformat()

    return {'Player_Name': self.player_name, 'First_Number': self.first_number, 
            'Second_Number': self.second_number, 'Solution': self.solution, 
            'Player_Answer': self.player_answer, 'Operator': self.operator, 
            'Seconds': self.seconds, 'Correct': self.correct, 'Testing': self.test,
            'Date': self.date, 'Prediction': self.prediction}



################### New Code #######################

def testing_mode():

  def player_setup():
    """Getting and returning the name of the user. The user must have played
    the normal verision of the game before being accepted to testing mode."""

    print('Welcome to testing mode!')

    while True:

      name = input('Enter your Little Professor Player Name to begin: ')

      if file_check(player_settings_storage):
        df = retrieve_from_file(player_settings_storage)

        if name in df['Player_Name'].tolist():
          return name
          
        else:
          print("Player has not played Littler Professor before")
      else:
        print("Please play Little Professor, before accessing testing mode.")
        sys.exit()

  def test_question():
    for _ in range(2):
      create_problem([5], ['na'], random.choice(['+', '-', '*', '/']))
      problem.test = 'Yes'
      record_to_file(game_results_storage, problem.export(), game_results_fieldnames)

  global player
  player = Player(Player_Name = player_setup())

  round = 0
  while True:
    if round > 0:
      multi_round = input('Would you like another round of questions? ')
      if multi_round.lower() == 'yes':
        test_question()
      else:
        sys.exit()
    else:
      test_question()
      round += 1


  print('This concludes testing mode!')

#################### New Code Above ###################

if __name__ == "__main__":
    main()

Overwriting professor.py


In [20]:
%run professor.py testing

Welcome to testing mode!
Enter your Little Professor Player Name to begin: Test
5 + 1 = cat
Response must be numerical
5 + 1 = 6
20 / 10 = 2
Would you like another round of questions? no


In [15]:
%run professor.py

Welcome to Little Professor! A math skill building game.
Enter your player name, or if you are new here type New: Test
Welcome Back
Would you like to use the settings from your last game? (Y or N)Y
7 * 5 = cat
Response must be numerical
7 * 5 = dog
Response must be numerical
7 * 5 = 0
EEE
7 * 5 = 35
2 * 6 = bat
Response must be numerical
2 * 6 = bar
Response must be numerical
2 * 6 = 12
Score: 1


In [18]:
%run professor.py testing mode

Incorrect Argument(s)


In [ ]:
main()

Incorrect Argument(s)


In [16]:
test_df = retrieve_from_file(game_results_storage)
updated_test_df = retrieve_from_file(player_settings_storage)
updated_test_df

,Player_Name,Games_Played,Level,Focus,Rounds,Operator
0,Brian,46,[1],"[4, 5, 7]",5,-
1,Test,15,[1],"[4, 5, 6]",2,*
2,Test2,2,[1],['na'],1,-
3,Test3,2,"[4, 1, 5]",['na'],3,+
4,Test4,1,[1],"[3, 6, 8]",1,*
5,Test1,9,"[4, 0, 5]","[0, 1]",5,/
6,vi,2,"[4, 1, 9]",['na'],5,*


In [17]:
test_df

,Player_Name,First_Number,Second_Number,Solution,Player_Answer,Operator,Seconds,Correct,Testing,Date,Prediction
0,Brian,4,4,0,0,-,1.7,Yes,,2023-01-06,
1,Brian,3,5,-2,-2,-,3.71,Yes,,2023-01-06,
2,Brian,2,4,-2,-2,-,2.32,Yes,,2023-01-06,
3,Test,5,5,25,25,*,3.17,Yes,,2023-01-07,
4,Test,6,6,36,36,*,2.95,Yes,,2023-01-07,
...,...,...,...,...,...,...,...,...,...,...,...
162,Test1,4,1,4,4,/,1.13,Yes,,2023-01-15,
163,Test,8,5,40,40,*,2.45,Yes,,2023-01-15,
164,Test,8,4,32,32,*,3.07,Yes,,2023-01-15,
165,Test,7,5,35,0,*,7.14,No,,2023-01-15,


In [ ]:
# creating a testing mode
# testing mode will produce random operators and numbers for creation of problems for users
# testing mode will also update the problem class and show that this problem was produced randomly

def testing_mode():

  def player_setup():
    """Getting and returning the name of the user. The user must have played
    the normal verision of the game before being accepted to testing mode."""

    print('Welcome to testing mode!')

    while True:

      name = input('Enter your Little Professor Player Name to begin: ')

      if file_check(player_settings_storage):
        df = retrieve_from_file(player_settings_storage)

        if name in df['Player_Name'].tolist():
          return name
          
        else:
          print("Player has not played Littler Professor before")
      else:
        print("Please play Little Professor, before accessing testing mode.")
        sys.exit()

  def test_question():
    for _ in range(2):
      create_problem([5], ['na'], random.choice(['+', '-', '*', '/']))
      problem.test = 'Yes'
      record_to_file(game_results_storage, problem.export(), game_results_fieldnames)

  global player
  player = Player(Player_Name = player_setup())

  round = 0
  while True:
    if round > 0:
      multi_round = input('Would you like another round of questions (Yes/No)? ')
      if multi_round.lower() == 'yes':
        test_question()
      else:
        sys.exit()
    else:
      test_question()
      round += 1


  print('This concludes testing mode!')

In [12]:
# function to restrict answers to ints
def ask_question(x,y,sign):
  if sign != '/':
    while True:
      try:
        user_answer = int(input(f"{x} {sign} {y} = "))
        return user_answer

      except ValueError:
        print("Response must be numerical")

  else:
    while True:
      try:
        user_answer = int(input(f"{x * y} {sign} {y} = "))
        return user_answer
        
      except ValueError:
        print("Response must be numerical")


In [13]:
%%writefile professor.py

import random, csv, os, time, datetime, sys
import pandas as pd

# variables for file storage
game_results_storage = "little_professor_game_resutls.csv"
player_settings_storage = "player_settings.csv"
FOLDER = "/content/drive/MyDrive/Data Science Projects/Math_Skill_Builder/Data/"

# creating folder for storage
os.makedirs(FOLDER, exist_ok = True)

game_results_fieldnames = ['Player_Name', 'First_Number', 'Second_Number', 
                           'Solution', 'Player_Answer', 'Operator', 'Seconds', 
                           'Correct','Testing', 'Date', 'Prediction']
player_settings_fieldnames = ['Player_Name', 'Games_Played', 'Level', 'Focus', 
                              'Rounds', 'Operator']

def main():

  game_mode = commandline()

  if game_mode == 'testing':
    testing_mode()

  if game_mode == 'normal':


    welcome()

    final_score = 0

    for _ in range(player.rounds):
        details = create_problem(player.level, player.focus, player.operator)
        record_to_file(game_results_storage, problem.export(), game_results_fieldnames)
        score = check_solution(*details)
        final_score += score

    print(f'Score: {final_score}')

def welcome():
  global player
  print('Welcome to Little Professor! A math skill building game.')
  player_name = input('Enter your player name, or if you are new here type New: ')

  if player_name.lower() == 'new':
    player = Player.get()
    record_to_file(player_settings_storage, player.export(), player_settings_fieldnames)
    return

  if file_check(player_settings_storage):
    df = retrieve_from_file(player_settings_storage)
    if player_name in df['Player_Name'].tolist():
      print("Welcome Back")
      used_last_df = retrieve_from_file(player_settings_storage)
      player_set = used_last_df[used_last_df['Player_Name'] == player_name].sort_index(ascending = False)    

      sorted_dict = player_set.to_dict('records')
      converted = number_convert(sorted_dict[0])
      while True:
        used_last = input('Would you like to use the settings from your last game? (Y or N)')
        if used_last.lower() == 'y':
          player = Player(**converted)
          update_dataframe(player.export())
          return
        if used_last.lower() == 'n':
          player = Player(Player_Name = player_name, Games_Played = converted['Games_Played'])
          player.reset_settings()
          update_dataframe(player.export())
          return

    else:
      no_name()

  else:
    no_name()

def no_name():
  print("We did not recognize your name. Let me ask you some questions and get you started!")
  global player
  player = Player.get()
  record_to_file(player_settings_storage, player.export(), player_settings_fieldnames)

def get_name():
  while True:
    name = input('Name: ')

    if file_check(player_settings_storage):
      df = retrieve_from_file(player_settings_storage)

      if name in df['Player_Name'].tolist():
        print('Another player has that name, choose another.')
      else:
        return name
    else:
      return name


def get_level():
    print('Type 1 for single, 2 for double, or 3 for triple digit numbers!')
    print('Or 4 to customize your number range.')
    start_settings = []
    while True:
        try:
            level = int(input('Level: '))
            if level in [1, 2, 3, 4]:
              break
        except ValueError:
            pass
    start_settings.append(level)            
    if start_settings[0] == 4:
        while True:
          try:
            num_range = input('Number range X-Y: ').split('-')
            cus_x = int(num_range[0])
            start_settings.append(cus_x)
            cus_y = int(num_range[1])
            start_settings.append(cus_y)
            break
          except ValueError:
            pass
    return start_settings

def get_focus():
  focus = []
  while True:
    focus_check = input("Would you like to focus on specific numbers? (Y or N) ")
    if focus_check.lower() == 'y':
      while True:
        try:
          numbers = input("What number(s) would you like to focus on? (ex. 1,3,5) ").split(',')
          for num in numbers:
            num = num.strip(' ')
            focus.append(int(num))
          return focus
        except:
          pass

    if focus_check.lower() == 'n':
      focus.append('na')
      return focus

def get_rounds():
  while True:
    try:
      rounds = int(input('How many questions would you like? '))
      return rounds
    except ValueError:
      pass

def get_operator():
  while True:
    try:
      operator = input("Pick your operator [+, -, *, /]: ")
      if operator in ['+', '-', '*', '/']:
        return operator
    except ValueError:
      pass


def generate_integer(level):
    if level[0] == 1:
        return random.randint(0, 9)
    if level[0] == 2:
        return random.randint(10, 99)
    if level[0] == 3:
        return random.randint(100, 999)
    if level[0] == 4:
        return random.randint(level[1], level[2])
    if level[0] == 5:
        return random.randint(0, 15)

def create_problem(level, focus, operator):
    global problem
    problem = Problem(Player_Name=player.name)

    x = generate_integer(level)
    if focus[0] == 'na':
      y = generate_integer(level)
    else:
      y = random.choice(focus)

    # filter to prevent y from becoming 0 when dividing
    while operator == '/' and y == 0:
      if focus[0] == 'na':
        y = generate_integer(level)
      else:
        y = random.choice(focus)

    if operator != '/':
      time_start = time.time()
      answer = ask_question(x,y,operator)
      time_stop = time.time()

      if operator == '+':
        ans = x + y
      elif operator == '-':
        ans = x - y
      elif operator == '*':
        ans = x * y

      problem.first_number = x
      problem.second_number = y
      problem.solution = ans
      problem.player_answer = answer
      problem.operator = operator
      problem.seconds = round(time_stop - time_start, 2)
      return [x, y, ans, answer, operator]
      
    else:
      time_start = time.time()
      answer = ask_question(x,y,operator)
      time_stop = time.time()

      product = x * y
      ans = x

      problem.first_number = product
      problem.second_number = y
      problem.solution = ans
      problem.player_answer = answer
      problem.operator = operator
      problem.seconds = round(time_stop - time_start, 2)

      return [product, y, ans, answer, operator]

def check_solution(first, second, solution, response, operator):
    tries = 0

    if solution == response:
        return 1
    else:
        while tries < 2:
            print('EEE')
            again = int(input(f'{first} {operator} {second} = '))
            if again == solution:
                return 0
            tries += 1

    print('EEE')
    print(f'Correct answer: {first} {operator} {second} = {solution}')
    return 0

def ask_question(x,y,sign):
  if sign != '/':
    while True:
      try:
        user_answer = int(input(f"{x} {sign} {y} = "))
        return user_answer

      except ValueError:
        print("Response must be numerical")

  else:
    while True:
      try:
        user_answer = int(input(f"{x * y} {sign} {y} = "))
        return user_answer
        
      except ValueError:
        print("Response must be numerical")


# function to convert strings to integers after retrieval from dataframe

def number_convert(dict_):
  converted_level = []
  converted_focus = []
  convert = dict_

  level_list = dict_['Level']
  if dict_['Level'].startswith('['):
    level_list = level_list.strip('[')
    level_list = level_list.strip(']')
    level_list = level_list.split(',')

    for _ in level_list:
      _.replace(' ', '')
      converted_level.append(int(_))
  convert['Level'] = converted_level

  focus_list = dict_['Focus']
  if dict_['Focus'].startswith('['):
    focus_list = focus_list.strip('[')
    focus_list = focus_list.strip(']')
    focus_list = focus_list.strip("'")
    focus_list = focus_list.split(',')

    if focus_list == ['na']:
      convert['Focus'] = focus_list
    else:
      for _ in focus_list:
        _.replace(' ', '')
        converted_focus.append(int(_))
      convert['Focus'] = converted_focus

  round = dict_['Rounds']
  convert['Rounds'] = int(round)

  game = dict_['Games_Played']
  convert['Games_Played'] = int(game)

  return convert

# creating a function to check if a file exsits
def file_check(file):
  folderfile = FOLDER + file
  file_exists = os.path.isfile(folderfile)
  if file_exists:
    return True
  else:
    return False

def record_to_file(file, results, fieldnames, edit = 'a'):
  folderfile = FOLDER + file
  if file_check(file):
    with open(folderfile, edit, newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      if edit == 'w':
        writer.writeheader()
        for row in results:
          writer.writerow(row)
      else:
        writer.writerow(results)

  else:
    with open(folderfile, 'w', newline = '') as f:
      writer = csv.DictWriter(f, fieldnames = fieldnames)
      writer.writeheader()
      writer.writerow(results)

def retrieve_from_file(file):
  folderfile = FOLDER + file
  with open(folderfile, 'r') as f:
    reader = csv.DictReader(f)
    results_df = pd.DataFrame(reader)
  return results_df


def update_dataframe(info):
  df = retrieve_from_file(player_settings_storage)
  players_index = df.index[df['Player_Name'] == info['Player_Name']]
  index_num = list(players_index).pop(0)
  for key, value in info.items():
    df.at[index_num, key] = value

  df_dictionaries = df.to_dict('records')
  record_to_file(player_settings_storage, df_dictionaries, player_settings_fieldnames, 'w')


def commandline():
  """ This funciton with check the command line and determine if the normal game
  is played, or the testing mode. The function will return 'normal' or 'testing'
  based off command line arguments"""

  if len(sys.argv) == 1:
    return 'normal'
  elif len(sys.argv) == 2:
    if sys.argv[1] == 'testing':
      return 'testing'
    else:
      print('Incorrect Mode')
  else:
    print('Incorrect Argument(s)')


class Player:

  def __init__(self, Player_Name, Level = 0, Focus = 0, Rounds = 0, Operator = '', Games_Played = 0):
    self.name = Player_Name
    self.level = Level
    self.focus = Focus
    self.rounds = Rounds
    self.operator = Operator
    self.games_played = Games_Played + 1

  def __str__(self):
    ...
  
  @classmethod
  def get(cls):
    name = get_name()
    level = get_level()
    focus = get_focus()
    rounds = get_rounds()
    operator = get_operator()
    return cls(name, level, focus, rounds, operator)

  def reset_settings(self):
    self.level = get_level()
    self.focus = get_focus()
    self.rounds = get_rounds()
    self.operator = get_operator()

  def export(self):
    return {'Player_Name': self.name, 'Level': self.level, 'Focus': self.focus, 
            'Rounds': self.rounds, 'Operator': self.operator, 
            'Games_Played': self.games_played}

# creating a game results class to store and retrieve all information

class Problem:

  def __init__(self, Player_Name = None, First_Number = None, Second_Number = None, Solution = None,
               Player_Answer = None, Operator = None, Seconds = None, Correct = None,
               Testing = None, Prediction = None):
    self.player_name = Player_Name
    self.first_number = First_Number
    self.second_number = Second_Number
    self.solution = Solution
    self.player_answer = Player_Answer
    self.operator = Operator
    self.seconds = Seconds
    self.correct = Correct
    self.test = Testing
    self.prediction = Prediction


  def export(self):
    if self.solution == self.player_answer:
      if self.seconds < 7:
        self.correct = 'Yes'
      else:
        self.correct = 'No'
    else:
      self.correct = 'No'

    self.date = datetime.date.today().isoformat()

    return {'Player_Name': self.player_name, 'First_Number': self.first_number, 
            'Second_Number': self.second_number, 'Solution': self.solution, 
            'Player_Answer': self.player_answer, 'Operator': self.operator, 
            'Seconds': self.seconds, 'Correct': self.correct, 'Testing': self.test,
            'Date': self.date, 'Prediction': self.prediction}



################### New Code #######################

def testing_mode():

  def player_setup():
    """Getting and returning the name of the user. The user must have played
    the normal verision of the game before being accepted to testing mode."""

    print('Welcome to testing mode!')

    while True:

      name = input('Enter your Little Professor Player Name to begin: ')

      if file_check(player_settings_storage):
        df = retrieve_from_file(player_settings_storage)

        if name in df['Player_Name'].tolist():
          return name
          
        else:
          print("Player has not played Littler Professor before")
      else:
        print("Please play Little Professor, before accessing testing mode.")
        sys.exit()

  def test_question():
    for _ in range(2):
      create_problem([5], ['na'], random.choice(['+', '-', '*', '/']))
      problem.test = 'Yes'
      record_to_file(game_results_storage, problem.export(), game_results_fieldnames)

  global player
  player = Player(Player_Name = player_setup())

  round = 0
  while True:
    if round > 0:
      multi_round = input('Would you like another round of questions? ')
      if multi_round.lower() == 'yes':
        test_question()
      else:
        sys.exit()
    else:
      test_question()
      round += 1


  print('This concludes testing mode!')

#################### New Code Above ###################

if __name__ == "__main__":
    main()

Overwriting professor.py
